# Liputan 6
## Klasifikasi jenis berita berdasarkan judul

In [160]:
# install libarary
! pip install pandas
! pip install numpy
! pip install nltk
! pip install Sastrawi
! pip install sklearn


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for sklearn: started
  Running setup.py install for sklearn: finished with status 'done'


  DEPRECATION: sklearn is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559


In [270]:
# import library

import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
# nltk.download()

In [194]:
# load data

df = pd.read_csv("raw.csv")
df.head()

,Judul,Jenis,User Aktif
0,Penyebab Wanita Gagal Raih Orgasme Saat Bercin...,Health,9
1,6 Potret Perayaan Ulang Tahun Okie Agustina ke...,Hot,7
2,Bacaan Doa Setelah Sholat Tahajud Rasulullah S...,Islami,7
3,Ternyata Pihak-Pihak Ini yang Disebut Punya Bi...,Crypto,6
4,Rekomendasi Film-Film Seru di Bioskop Sepanjan...,Regional,6


In [195]:
# memilih kolom-kolom yang akan di drop 

to_drop = ['User Aktif']

# melakukan drop pada data yang dipilih

df.drop(to_drop, inplace=True, axis=1)

# Hasil penghilenang kolom
df.head()

,Judul,Jenis
0,Penyebab Wanita Gagal Raih Orgasme Saat Bercin...,Health
1,6 Potret Perayaan Ulang Tahun Okie Agustina ke...,Hot
2,Bacaan Doa Setelah Sholat Tahajud Rasulullah S...,Islami
3,Ternyata Pihak-Pihak Ini yang Disebut Punya Bi...,Crypto
4,Rekomendasi Film-Film Seru di Bioskop Sepanjan...,Regional


In [196]:
# melihat statistik dasar 

df.describe()

,Judul,Jenis
count,1350,1357
unique,1296,35
top,Rekam Jejak Fallincy Mbak Topi Subak Musuh Ber...,Hot
freq,5,356


In [197]:
# menghilangkan data judul yang sama
df.drop_duplicates(subset="Judul", keep=False, inplace=True)
df.describe()

,Judul,Jenis
count,1249,1249
unique,1249,29
top,Penyebab Wanita Gagal Raih Orgasme Saat Bercin...,Hot
freq,1,319


# Case folding

In [198]:
# mengubah ke lower case
df['Judul'] = df['Judul'].str.lower()
df['Jenis'] = df['Jenis'].str.lower()

df.head()

,Judul,Jenis
0,penyebab wanita gagal raih orgasme saat bercin...,health
1,6 potret perayaan ulang tahun okie agustina ke...,hot
3,ternyata pihak-pihak ini yang disebut punya bi...,crypto
4,rekomendasi film-film seru di bioskop sepanjan...,regional
5,manfaat sholat istikharah tata cara niat doa d...,regional


In [199]:
# menghilangkan karakter special dan numerik
df['Judul'] = df['Judul'].str.replace('\d*', '')
df['Judul'] = df['Judul'].str.replace('[^\w\s]', ' ')
df['Judul'] = df['Judul'].str.strip()
df.head()

C:\Users\AKU\AppData\Local\Temp\ipykernel_3356\2487329922.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Judul'] = df['Judul'].str.replace('\d*', '')
C:\Users\AKU\AppData\Local\Temp\ipykernel_3356\2487329922.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Judul'] = df['Judul'].str.replace('[^\w\s]', ' ')


,Judul,Jenis
0,penyebab wanita gagal raih orgasme saat bercin...,health
1,potret perayaan ulang tahun okie agustina ke ...,hot
3,ternyata pihak pihak ini yang disebut punya bi...,crypto
4,rekomendasi film film seru di bioskop sepanjan...,regional
5,manfaat sholat istikharah tata cara niat doa d...,regional


# Tokenizing and Stemming

In [200]:
# stemming and tokenizing function to sastrawati
def funSastrawati(val):
    stop_factory = ArrayDictionary(StopWordRemoverFactory().get_stop_words())
    str = StopWordRemover(stop_factory)
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    word = stemmer.stem(str.remove(val))
    return word
# Apply stemming and tokenizing
df['Judul'] = df['Judul'].apply(funSastrawati)
df.head()

,Judul,Jenis
0,sebab wanita gagal raih orgasme cinta salah sa...,health
1,potret raya ulang tahun okie agustina rias bik...,hot
3,nyata pihak pihak yang sebut punya bitcoin ban...,crypto
4,rekomendasi film film seru bioskop panjang sep...,regional
5,manfaat sholat istikharah tata cara niat doa w...,regional


In [235]:
df['Judul'] = df['Judul'].astype('string')
df['Jenis'] = df['Jenis'].astype('category')
df.dtypes

Judul      string
Jenis    category
dtype: object

In [277]:
coba = df
grouped = df.groupby('Jenis').size()
# axis = grouped.plot(kind='bar')
# plt.show()
for name, show in zip(grouped.index, grouped):
    if(show < 2):
        coba.drop(coba.loc[coba['Jenis'] == name].index, inplace=True)
coba.groupby('Jenis').size()

Jenis
asian            0
bisnis          53
bola            28
cek             11
citizen6       103
crypto           8
dan              0
disabilitas      2
fashion          6
global          57
health         163
hot            319
indonesia        0
islami          34
jateng           3
jatim            4
kabar           54
lifestyle       94
news            38
otomotif        30
photo            8
pileg            0
pilkada          0
regional        44
saham            3
showbiz        107
surabaya         8
tekno           62
video            5
dtype: int64

In [278]:
# tf df
tf = TfidfVectorizer()
kata = tf.fit_transform(coba['Judul'].astype('U'))
kata

<1244x3256 sparse matrix of type '<class 'numpy.float64'>'
	with 9348 stored elements in Compressed Sparse Row format>

In [279]:
# Spliting data 

X_train, X_test, y_train, y_test = train_test_split(kata, df['Judul'], test_size=0.2, random_state = 42)

In [280]:

scaler = StandardScaler()
# Fit only on X_train
scaler.fit(X_train)

# Scale both X_train and X_test
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

ValueError: Cannot center sparse matrices: pass `with_mean=False` instead. See docstring for motivation and alternatives.